<table align="left">
  <td>
    <a href="https://colab.research.google.com/drive/1rvJUY3GLe43pTxsh9VpllFfmIP9ddOLl" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# **Modelos de lenguaje**

In [1]:
!pip install -q transformers
!pip install -q sentencepiece

import transformers
from transformers import pipeline
import torch

# Comprobar si hay una GPU disponible
device = 0 if torch.cuda.is_available() else -1

# https://huggingface.co/docs/transformers/main_classes/logging
transformers.utils.logging.set_verbosity_error()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00


## Modelos de lenguaje estocásticos

In [2]:
texto = """
El perro salta sobre el muro.
El gato duerme en el sofá.
El niño juega en el parque.
La niña canta en el coro.
El perro corre tras el gato.
La niña lee un libro.
El niño dibuja un perro.
"""

# Limpiamos y tokenizamos el texto
texto = texto.lower().replace('\n', ' ').replace('.', '').strip()
palabras = texto.split(' ')

In [ ]:
from collections import defaultdict

bigramas = defaultdict(int)

for i in range(len(palabras) - 1):
    bigrama = (palabras[i], palabras[i+1])
    bigramas[bigrama] += 1

In [ ]:
def predecir_siguiente_palabra(palabra, bigramas):
    siguientes_palabras = {k[1]: v for k, v in bigramas.items() if k[0] == palabra}
    if not siguientes_palabras:
        return "No se puede predecir."

    palabra_predicha = max(siguientes_palabras, key=siguientes_palabras.get)
    return palabra_predicha

# Ejemplos de uso
print(predecir_siguiente_palabra('el', bigramas))  # Debería mostrar 'perro', 'gato', 'niño' o 'niña' dependiendo de las frecuencias
print(predecir_siguiente_palabra('la', bigramas))  # Debería mostrar 'niña'
print(predecir_siguiente_palabra('niña', bigramas))  # Debería mostrar 'canta' o 'lee' dependiendo de las frecuencias

perro
niña
canta


## Modelos de lenguaje basados en deep learning


### Análisis de sentimientos

In [ ]:
from transformers import pipeline

nlp = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

# Analizar el sentimiento del texto
result = nlp("No me importa nada")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

# Analizar el sentimiento del texto
result = nlp("Estoy loco por ti")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")


label: 1 star, with score: 0.712
label: 5 stars, with score: 0.3148


### Traducción automática

In [ ]:
from transformers import pipeline

# https://github.com/facebookresearch/flores/blob/main/flores200/README.md#languages-in-flores-200
nlp = pipeline("translation", model="facebook/nllb-200-distilled-600M", src_lang="spa_Latn", tgt_lang="eng_Latn", device=device)

# traducción del texto
result = nlp("No me importa nada")
print(result)


[{'translation_text': "I don't care about anything"}]


### Clasificación de tokens

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

nlp = pipeline("token-classification", model="xlm-roberta-large-finetuned-conll03-english", device=device)

result = nlp("El perro del tío Braulio no es un perro, es un demonio.")
print(result)


[{'entity': 'I-PER', 'score': 0.9999851, 'index': 7, 'word': '▁Bra', 'start': 17, 'end': 20}, {'entity': 'I-PER', 'score': 0.9999745, 'index': 8, 'word': 'ulio', 'start': 20, 'end': 24}]


### Resúmenes de textos

In [ ]:
nlp = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum", device=device)

texto="""
Los Sesgos en la Inteligencia Artificial: Un Asunto Urgente
Introducción
La inteligencia artificial (IA) ha llegado a ser parte integral de nuestras vidas, desde recomendaciones de productos en línea hasta diagnósticos médicos. Sin embargo, a medida que la IA se integra más en nuestros sistemas sociales y económicos, el problema del sesgo en estos modelos se ha convertido en un tema cada vez más preocupante.

¿Qué es el Sesgo en la IA?
El sesgo en la IA es la tendencia de un modelo a hacer generalizaciones erróneas basadas en los datos de entrenamiento. Estas generalizaciones pueden reflejar y perpetuar prejuicios humanos, ya sean raciales, de género, socioeconómicos o de otro tipo. Es decir, la IA no es intrínsecamente justa o injusta, sino que reproduce los sesgos que ya existen en la sociedad.

¿Cómo se Introduce el Sesgo?
El sesgo puede introducirse de varias maneras en los modelos de IA:

Datos de Entrenamiento: Si los datos usados para entrenar el modelo contienen sesgos, el modelo aprenderá esos sesgos.
Diseño del Modelo: Los algoritmos también pueden introducir sesgos si están diseñados de manera que priorizan ciertas características.
Interacción Humana: Los usuarios pueden, consciente o inconscientemente, alimentar el sistema con datos sesgados.
Casos Reales de Sesgo en la IA
Un ejemplo claro es el de los sistemas de reconocimiento facial que tienen dificultades para identificar a personas con tonos de piel más oscuros. También hay casos de chatbots que han aprendido lenguaje ofensivo y discriminatorio de las interacciones en línea. En ambos casos, las consecuencias del sesgo pueden ser graves.

Consecuencias del Sesgo
Las implicaciones del sesgo en la IA son inmensas, desde la discriminación en el acceso a servicios y oportunidades, hasta la injusticia en sistemas legales y penales. En el peor de los casos, los sesgos de IA pueden reforzar círculos viciosos de desigualdad.

Mitigando el Sesgo
Es fundamental que los ingenieros, los responsables políticos y otros interesados trabajen juntos para minimizar los sesgos. Algunas estrategias incluyen la revisión ética, la auditoría de algoritmos, y la adopción de técnicas de "aprendizaje justo".

Conclusión
El sesgo en la IA es un problema crítico que requiere una atención inmediata y colaborativa. A medida que la IA sigue evolucionando, es fundamental abordar este problema de frente para asegurar que la tecnología actúe como una fuerza para el bien, en lugar de perpetuar las desigualdades existentes.
"""

resumen = nlp(texto)

print(resumen)

del nlp


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[{'summary_text': 'El sesgo en la inteligencia artificial (IA) es un problema crítico que requiere atención inmediata y colaborativa.'}]


### Un chatbot

In [ ]:
from transformers import pipeline, Conversation

chatbot = pipeline("conversational", model="facebook/blenderbot-400M-distill")

while True:
    user = Conversation(input("You: "))
    response = chatbot([user])
    print("Chatbot:", response)


Chatbot: Conversation id: 960c07cb-3009-4c41-90fb-0541e0c34c2f
user: el niño
assistant:  El niro is an Italian actor, director, and producer. Have you seen any of his movies?

